In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import xgboost as xgb
from fastai.vision.all import *
from ENSEMBLE_SCORING_ALGORITHM import *

C:\Users\mindf\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Initialize all Gradient Boosted Trees
intialize_trees()

In [9]:
#Import Generator
from GENERATOR_HEALTHCARE_MODEL import LeverGenerator

#Generator model
generator_model = LeverGenerator(1200)
#Configure loss function
loss_func = nn.MSELoss()
#Configure Optimizer
optimizer = torch.optim.Adam(generator_model.parameters(), lr = 0.01)

#Training script for PyTorch (training and validation loops)
def train_model(epochs, generator):
    train_accuracies = []
    train_losses = []
    valid_accuracies = []
    valid_losses = []
    #Training Loop
    iteration_loop(training = True, epochs = epochs, generator = generator, losses = train_losses, accuracies = train_accuracies)
    #Validation Loop
    iteration_loop(training = False, epochs = epochs, generator = generator, losses = valid_losses, accuracies = valid_accuracies)
    return train_accuracies, train_losses

def iteration_loop(training, epochs, generator, losses, accuracies):
    epoch_losses = []
    total_loss = 0.0
    for epoch in range(epochs): 
        #Set gradients to zero
        generator.zero_grad()
        inputs = torch.rand((1, 1200))
        output = generator(inputs)
        #Air quality metrics
        aq_metrics = output[:, :13]
        exercise_metrics = output[:, 14:16]
        #Blood report metrics
        blood_report_metrics = output[:, 17:31]
        #Nutrition metrics
        nutrition_metrics = output[:, 32:37]
        #Sleep metrics
        sleep_metrics = output[:, 37:40]
        METRICS = {'pollution_gradient_tree' : aq_metrics, 'exercise_gradient_tree': exercise_metrics,
                   'nutrition_gradient_tree': nutrition_metrics, 'sleep_gradient_tree': sleep_metrics, 
                   'blood_gradient_tree': blood_report_metrics}
        avg_score = torch.tensor(get_avg_model_score(METRICS), requires_grad = True)
        loss = torch.sub(1.0,  )
        if not training:
            loss.backward()
        total_loss += loss 
        if epoch == epochs - 1:
            avg_loss = total_loss / epochs
            print("TRAINING [{}] LOSS: ", avg_loss)    

In [10]:
#Train the model
train_model(epochs = 10, generator = generator_model)

OUTPUT tensor([[-0.0004, -0.0072, -0.0198,  0.0343, -0.0487, -0.0063,  0.0332,  0.0463,
         -0.0112, -0.0007, -0.0124,  0.0395, -0.0655,  0.0110, -0.0229, -0.0047,
          0.0998,  0.0032, -0.0016, -0.0164,  0.0455,  0.0472, -0.0448, -0.0195,
         -0.0797,  0.0683,  0.0339,  0.0038,  0.0687,  0.0653, -0.0083,  0.0369,
         -0.0440,  0.0365,  0.0013,  0.0472, -0.0143,  0.0232,  0.0050, -0.0134]],
       grad_fn=<AddmmBackward>)
AQ:  tensor([[-0.0004, -0.0072, -0.0198,  0.0343, -0.0487, -0.0063,  0.0332,  0.0463,
         -0.0112, -0.0007, -0.0124,  0.0395, -0.0655]],
       grad_fn=<SliceBackward>)
EX  tensor([[-0.0229, -0.0047]], grad_fn=<SliceBackward>)
BR  tensor([[ 0.0032, -0.0016, -0.0164,  0.0455,  0.0472, -0.0448, -0.0195, -0.0797,
          0.0683,  0.0339,  0.0038,  0.0687,  0.0653, -0.0083]],
       grad_fn=<SliceBackward>)
NU  tensor([[-0.0440,  0.0365,  0.0013,  0.0472, -0.0143]],
       grad_fn=<SliceBackward>)
METRICS  tensor([[-0.0004, -0.0072, -0.0198,  0.

([], [])